# Queries with and without Azure OpenAI

Now that we have our Search Engine loaded and running, we are going to try some example queries and then use Azure OpenAI service to see if we can get even better results

## Set up variables

In [6]:
api_version = '2021-04-30-Preview'

# endpoint = os.environ.get("SEARCH_ENDPOINT")
# api_key = os.getenv("SEARCH_KEY")

endpoint = "https://azure-cog-search-pabdyosydd7ta.search.windows.net"
api_key = "DDDUwtXOCSOjm1fPBRLNFofKEEvxXDpGF0Sy4S3ktjAzSeAgbz9Q"
index_name = "cogsrch-index"

In [9]:
# Setup the Payloads header
api_version = '2021-04-30-Preview'
headers = {'Content-Type': 'application/json','api-key': api_key}
params = {'api-version': api_version}

## Without Azure OpenAI

In [15]:
QUESTION = "what is reinforcment lerning" # Notice that is misspelled intentionally

In [13]:
import urllib
import requests
from IPython.display import display, HTML

url = endpoint + '/indexes/'+ index_name + '/docs'
url += '?api-version={}'.format(api_version)
url += '&search={}'.format(QUESTION)
url += '&select=content'
url += '&$top=5'
url += '&queryLanguage=en-us'
url += '&queryType=semantic'
url += '&semanticConfiguration=my-semantic-config'
url += '&$count=true'
url += '&speller=lexicon'
url += '&answers=extractive|count-3'
url += '&captions=extractive|highlight-true'
url += '&highlightPreTag=' + urllib.parse.quote('<span style="background-color: #f5e8a3">', safe='')
url += '&highlightPostTag=' + urllib.parse.quote('</span>', safe='')

resp = requests.get(url, headers=headers)
print(url)
print(resp.status_code)

search_results = resp.json()
print("Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))
print("Highest Search Score: {}".format(search_results['value'][0]['@search.score']))

https://azure-cog-search-pabdyosydd7ta.search.windows.net/indexes/cogsrch-index/docs?api-version=2021-04-30-Preview&search=what is reinforcment lerning&select=content&$top=5&queryLanguage=en-us&queryType=semantic&semanticConfiguration=my-semantic-config&$count=true&speller=lexicon&answers=extractive|count-3&captions=extractive|highlight-true&highlightPreTag=%3Cspan%20style%3D%22background-color%3A%20%23f5e8a3%22%3E&highlightPostTag=%3C%2Fspan%3E
200
Results Found: 9792, Results Returned: 5
Highest Search Score: 24.307634


In [18]:
# Answers from semantic Search
display(HTML('<h3>Top Answers</h3>'))
for result in search_results['@search.answers']:
    if result['score'] > 0.5:
        display(HTML('<h5>' + 'Answer - score: ' + str(result['score']) + '</h5>'))
        display(HTML(result['highlights']))

        
# Results from semantic search
contents = dict()

print("\n\n")
display(HTML('<h3>Top Results</h3>'))
for result in search_results['value']:
    if result['@search.rerankerScore'] > 1:
        display(HTML('<h5>' + result['metadata_storage_name'] + '&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;score: ' + str(result['@search.rerankerScore']) + '</h5>'))
        for caption in result['@search.captions']:
            # print(caption)
            display(HTML(caption['highlights']))
        contents[result['metadata_storage_name']]=result['content']

### Comments on Query results

As seen above the semantic search feature of Azure Cognitive Search service is pretty good. It gives us the top answers and also the top results with the corresponding file and the paragraph where the answers is possible located
Let's see if we can make this better with Azure OpenAI

## Using Azure OpenAI